## METHODS of SIR Estimation    


In [ ]:
import numpy as np
import csv


# The actual SIR values are -10 dB
fft_data = np.genfromtxt('data.csv', delimiter=',', skip_header=1)

# Locate the peak of symbol 1
peak_index = np.argmax(fft_data[:,1])

# Receiver noise floor estimate
T = 298.16
k = 1.38e-23 # Boltzmann constant
BW = 250e3
noise_floor = k * T * BW 




# Calculate the SIR
with open("test.csv") as fp:
    reader = csv.reader(fp, delimiter=",", quotechar='"')
    # next(reader, None)  # skip the headers
    data_read = [row for row in reader]




